In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/drive/My Drive/clean_loan_data.csv")
df.head()

,STATE,BANK_STATE,APPROVAL_YEAR,TERM,EMPLOYEE_COUNT,JOBS_CREATED_COUNT,JOBS_RETAINED_COUNT,IS_URBAN,IS_REVOLVER,IS_LOW_DOC,...,INDUSTRY,IS_Franchise,NEW,DAYS_TO_DIBURSEMENT,DEFAULT,DisbursementYEAR,SAME_STATE,SBA_APPROVED_PERCENTAGE,REAL_ESTATE,DISBURSED_GREATER_THAN_LOAN
0,IN,OH,1997,84,4,0,0,0,0,1,...,Retail_trade,0,1,730,0,1999,0,0.80,0,0
1,IN,IN,1997,60,2,0,0,0,0,1,...,Accom/Food_serv,0,1,92,0,1997,1,0.80,0,0
2,IN,IN,1997,180,7,0,0,0,0,0,...,Healthcare/Social_assist,0,0,306,0,1997,1,0.75,0,0
3,CT,DE,1997,120,19,0,0,0,0,0,...,Manufacturing,0,0,122,0,1997,0,0.75,0,0
4,FL,AL,1997,84,1,0,0,0,0,1,...,Other_no_pub,0,1,487,0,1998,0,0.80,0,0


In [25]:
df = df.iloc[:10,:]
df.shape

(10, 32)

In [26]:
# Define the function to create a prompt
def create_prompt(data_point):
    prompt = f"""
    Predict the default probability for the following loan applicant:

    - State: {data_point['STATE']}
    - Term: {data_point['TERM']}
    - Disbursement Amount: {data_point['DISBURSEMENT_AMOUNT']}
    - Loan Amount: {data_point['LOAN_AMOUNT']}
    - Treasury Yield: {data_point['TREASURY_YIELD']}
    - CPI Index: {data_point['CPI_INDEX']}
    - GDP: {data_point['GDP']}
    - Mortage 30 US fixed: {data_point['MORTGAGE_30_US_FIXED']}
    - Industry : {data_point['INDUSTRY']}
    - Unrate: {data_point['UNRATE']}
    - INDPRO Index: {data_point['INDPRO_INDEX']}
    - UMCSENT Index: {data_point['UMCSENT_INDEX']}
    - CSUSHPINSA Index: {data_point['CSUSHPINSA_INDEX']}
    - CP Index: {data_point['CP_INDEX']}
    - Days to Disbursement: {data_point['DAYS_TO_DIBURSEMENT']}
    - Disbursement Year: {data_point['DisbursementYEAR']}
    - In same state as bank: {data_point['SAME_STATE']}
    - SBA Approved Percentage: {data_point['SBA_APPROVED_PERCENTAGE']}
    - Is it real estate: {data_point['REAL_ESTATE']}
    - Disbursed greater than loan: {data_point['DISBURSED_GREATER_THAN_LOAN']}
    """
    return prompt

# Create a list to store prompts and responses
data_list = []

# Generate prompts and responses
for index, row in df.iterrows():
    prompt = create_prompt(row)
    response = f"Default Probability: {row['DEFAULT']}"
    data_list.append((prompt, response))

In [27]:
data_list[0]

('\n    Predict the default probability for the following loan applicant:\n\n    - State: IN\n    - Term: 84\n    - Disbursement Amount: 60000.0\n    - Loan Amount: 60000.0\n    - Treasury Yield: 6.3525\n    - CPI Index: 160.525\n    - GDP: 8577.5525\n    - Mortage 30 US fixed: 7.598846153846154\n    - Industry : Retail_trade\n    - Unrate: 4.941666666666666\n    - INDPRO Index: 80.56905\n    - UMCSENT Index: 103.21666666666664\n    - CSUSHPINSA Index: 85.31741666666666\n    - CP Index: 592.26625\n    - Days to Disbursement: 730\n    - Disbursement Year: 1999\n    - In same state as bank: 0\n    - SBA Approved Percentage: 0.8\n    - Is it real estate: 0\n    - Disbursed greater than loan: 0\n    ',
 'Default Probability: 0')

In [28]:
from langchain.docstore.document import Document

documents = []

# Iterate over rows using .rows() method
for i, row_tuple in df.iterrows():
    document = f"id:{i}\Customer_Behavior: {row_tuple[1]}\Default_Status: {row_tuple[0]}"
    documents.append(Document(page_content=document))

<ipython-input-28-7c4b1ad0d187>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  document = f"id:{i}\Customer_Behavior: {row_tuple[1]}\Default_Status: {row_tuple[0]}"


In [7]:

!pip install langchain langchain-community langchain-core transformers

In [29]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hg_embeddings = HuggingFaceEmbeddings()

<ipython-input-29-733749dc74cb>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hg_embeddings = HuggingFaceEmbeddings()


In [30]:
!pip install chromadb

In [31]:
!pip install bitsandbytes

In [32]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

model_id = 'HuggingFaceH4/zephyr-7b-beta'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

cpu


In [33]:
!pip install accelerate

In [34]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [35]:
# Initialize the query pipeline with increased max_length
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_length=6000,  # Increase max_length
    max_new_tokens=500,  # Control the number of new tokens generated
    device_map="auto",
)

NameError: name 'model' is not defined

In [36]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

llm = HuggingFacePipeline(pipeline=query_pipeline)

question = "Please explain Default in Credit Risk?"
response = llm(prompt=question)

full_response =  f"Question: {question}\nAnswer: {response}"
display(Markdown(colorize_text(full_response)))

NameError: name 'query_pipeline' is not defined

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from IPython.display import display, Markdown
import os
import warnings
warnings.filterwarnings('ignore')


# Define the prompt template
template = """
You are an Credit Risk Expert in Financial Text Data, Analyse them and Predict is the Given Small Business is going to Default or not?. If you don't know the answer, just say "Sorry, I Don't Know."
Question: {question}
Context: {context}
Answer:
"""
PROMPT = PromptTemplate(input_variables=["context", "query"], template=template)

# Ensure llm and langchain_chroma are properly initialized
retriever = langchain_chroma.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": PROMPT}
)

# Define the question
question = "\n    Predict the default probability for the following loan applicant:\n\n    - State: CA\n    - Term: 90\n    - Disbursement Amount: 345720.0\n    - Loan Amount: 345720.0\n    - Treasury Yield: 6.3525\n    - CPI Index: 160.525\n    - GDP: 8577.5525\n    - Mortage 30 US fixed: 7.598846153846154\n    - Industry : Retail_trade\n    - Unrate: 4.941666666666666\n    - INDPRO Index: 80.56905\n    - UMCSENT Index: 103.21666666666664\n    - CSUSHPINSA Index: 85.31741666666666\n    - CP Index: 592.26625\n    - Days to Disbursement: 730\n    - Disbursement Year: 1999\n    - In same state as bank: 1\n    - SBA Approved Percentage: 0.8\n    - Is it real estate: 0\n    - Disbursed greater than loan: 0\n   "


# Run the QA chain
try:
    result = qa_chain({"query": question})
    display(result)
except RuntimeError as e:
    print(f"RuntimeError encountered: {e}")